# 14744 Homework (10 pts)

## Training Time Series Models

## **Due:** Wednesday March 15

## Submission Instructions

*   Save a copy of this notebook in your Google Drive by clicking File->Save a copy in Drive
*   Use the Python 3 programming language to complete the programming exercises in the provided code cells
*   Make sure that all your code and code output appear correctly
*   Submit **both** of the following copies of your notebook on Canvas
    - .pdf version (by printing your notebook to pdf)
    - .ipynb version (by clicking File->Download .ipynb)

### Starter code

To get started run the cell below by clicking its 'play' icon. If you want to run this cell a second time, you may need to reset the runtime by clicking Runtime->Factory reset runtime.

In [1]:
## DO NOT EDIT THIS CELL

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import os

# Download and unzip accelerometer trace data
!mkdir ./data/
!wget https://www.andrew.cmu.edu/user/dvaroday/14744/data.zip
!mv data.zip ./data
!unzip ./data/data.zip -d ./data
!rm ./data/data.zip

# Initialize paths and list of raw trace filenames
ground_truth_path = '/content/data/ground_truth/'

path = '/content/data/raw_traces/'
filenames = sorted(os.listdir(path))

# Define functions presented in lecture
def brush_indicator(filename, alpha, threshold):
  acceleration = np.genfromtxt(path+filename).astype(float)
  jerk = np.zeros(acceleration.shape)
  jerk[1:,:] = acceleration[1:,:] - acceleration[:-1,:]
  jerk_magnitude = np.sqrt(np.sum(jerk**2, axis=1))
  smoothed = np.zeros(jerk_magnitude.shape)
  smoothed[0] = jerk_magnitude[0]
  for i in range(1, len(smoothed)):
    smoothed[i] = alpha * jerk_magnitude[i] + (1-alpha) * smoothed[i-1]
  indicator = smoothed > threshold
  return indicator

def error_cost_function(filename, alpha, threshold):
  indicator = brush_indicator(filename, alpha, threshold)
  ground_truth = np.genfromtxt(ground_truth_path + 'Truth_' + filename)
  cost = np.sum(indicator != ground_truth)/len(indicator)
  return cost

mkdir: cannot create directory ‘./data/’: File exists
--2023-03-15 00:54:46--  https://www.andrew.cmu.edu/user/dvaroday/14744/data.zip
Resolving www.andrew.cmu.edu (www.andrew.cmu.edu)... 128.2.42.53
Connecting to www.andrew.cmu.edu (www.andrew.cmu.edu)|128.2.42.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94022 (92K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  91.82K   297KB/s    in 0.3s    

2023-03-15 00:54:47 (297 KB/s) - ‘data.zip’ saved [94022/94022]

Archive:  ./data/data.zip
replace ./data/ground_truth/Truth_Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./data/ground_truth/Truth_Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt  
  inflating: ./data/ground_truth/Truth_Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt  
  inflating: ./data/ground_truth/Truth_Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt  
  inflating: ./data/groun

### Problem 1 (3 pts)

In the cell below write a leave-one-out cross-validation procedure for the time series model described in `brush_indicator()`. Each iteration of training should optimize the model over all combinations of parameters $\alpha \in \{0.1, 0.15, 0.2\}$ and $\text{threshold} \in \{7, 8, 9\}$. Your code should print out an average testing error cost of `0.0140` as reported in lecture.

In [2]:
## EDIT THE CODE IN THIS CELL

alphas = [0.1, 0.15, 0.2]
thresholds = [7, 8, 9]

avg_testing_error_cost = 0

for i in range(len(filenames)):
  train_set = filenames[:i] + filenames[i + 1:]
  mincost = float('inf')
  mina, mint = float('inf'), float('inf')
  for a in alphas:
    for t in thresholds:
      costs = []
      for f in train_set:
        costs.append(error_cost_function(f, a, t))
      if sum(costs)/len(costs) < mincost:
        mincost = sum(costs)/len(costs)
        mina, mint = a, t
  avg_testing_error_cost += error_cost_function(filenames[i], mina, mint)

avg_testing_error_cost /= len(filenames)

print(avg_testing_error_cost)

0.01403845008161896


### Problem 2 (2 pts)

Now that you have validated that the `brush_indicator()` model is sound, train the model using all the traces. As before you should optimize the model over all combinations of parameters $\alpha \in \{0.1, 0.15, 0.2\}$ and $\text{threshold} \in \{7, 8, 9\}$. Your code should print out the optimal values of $\alpha$ and $\text{threshold}$

In [3]:
## EDIT THE CODE IN THIS CELL

alphas = [0.1, 0.15, 0.2]
thresholds = [7, 8, 9]
mincost = float('inf')
mina, mint = float('inf'), float('inf')
for a in alphas:
  for t in thresholds:
    costs = []
    for f in filenames:
      costs.append(error_cost_function(f, a, t))
    if sum(costs)/len(costs) < mincost:
      mincost = sum(costs)/len(costs)
      mina, mint = a, t
print('alpha =', mina)
print('threshold =', mint)

alpha = 0.15
threshold = 8


### Problem 3 (3 pts)

Complete the implementation of the `brush_indicator2()` model, so that it is identical to `brush_indicator()` except that it uses two thresholds as described in lecture.

Then write a leave-one-out cross-validation procedure for `brush_indicator2()`. Fix $\alpha$ to the value you determined in Problem 2. Each iteration of training should optimize the model over combinations of parameters $\text{threshold_lo}, \text{threshold_hi} \in \{7, 8, 9\}$. Your code should print out an average testing error cost of `0.0113` as reported in lecture.

In [4]:
## EDIT THE CODE IN THIS CELL

# Complete implementation of brush_indicator2() so that it uses two thresholds
def brush_indicator2(filename, alpha, threshold_lo, threshold_hi):
  acceleration = np.genfromtxt(path+filename).astype(float)
  jerk = np.zeros(acceleration.shape)
  jerk[1:,:] = acceleration[1:,:] - acceleration[:-1,:]
  jerk_magnitude = np.sqrt(np.sum(jerk**2, axis=1))
  smoothed = np.zeros(jerk_magnitude.shape)
  smoothed[0] = jerk_magnitude[0]
  for i in range(1, len(smoothed)):
    smoothed[i] = alpha * jerk_magnitude[i] + (1-alpha) * smoothed[i-1]
  indicator = np.zeros(smoothed.shape)

  indicator[0] = smoothed[0] > threshold_hi
  for i in range(1, indicator.shape[0]):
    if smoothed[i] > threshold_hi:
      indicator[i] = 1
    elif smoothed[i] < threshold_lo:
      indicator[i] = 0
    else:
      indicator[i] = indicator[i - 1]

  return indicator

def error_cost_function2(filename, alpha, threshold_lo, threshold_hi):
  indicator = brush_indicator2(filename, alpha, threshold_lo, threshold_hi)
  ground_truth = np.genfromtxt(ground_truth_path + 'Truth_' + filename)
  cost = np.sum(indicator != ground_truth)/len(indicator)
  return cost


# Write a leave-one-out cross-validation procedure for brush_indicator()
thresholds = [7, 8, 9]

avg_testing_error_cost = 0

for i in range(len(filenames)):
  train_set = filenames[:i] + filenames[i + 1:]
  mincost = float('inf')
  mina, mintlo, minthi = 0, 0, 0
  for a in alphas:
    for m in range(len(thresholds) - 1):
      for n in range(m + 1, len(thresholds)):
        costs = []
        for f in train_set:
          costs.append(error_cost_function2(f, a, thresholds[m], thresholds[n]))
        if sum(costs)/len(costs) < mincost:
          mincost = sum(costs)/len(costs)
          mina, mintlo, minthi = a, thresholds[m], thresholds[n]
  avg_testing_error_cost += error_cost_function2(filenames[i], mina, mintlo, minthi)

avg_testing_error_cost /= len(filenames)


print(avg_testing_error_cost)

0.011318174889272446


### Problem 4 (1 pt)

Train the `brush_indicator2()` model using all the traces. As before fix $\alpha$ to the value you determined in Problem 2, and optimize the model over combinations of parameters $\text{threshold_lo}, \text{threshold_hi} \in \{7, 8, 9\}$. Your code should print out the optimal values of $\text{threshold_lo}$ and $\text{threshold_hi}$.

In [5]:
## EDIT THE CODE IN THIS CELL

thresholds = [7, 8, 9]

train_set = filenames[:i] + filenames[i + 1:]
mincost = float('inf')
mina, mintlo, minthi = 0, 0, 0
for a in alphas:
  for m in range(len(thresholds) - 1):
    for n in range(m + 1, len(thresholds)):
      costs = []
      for f in filenames:
        costs.append(error_cost_function2(f, a, thresholds[m], thresholds[n]))
      if sum(costs)/len(costs) < mincost:
        mincost = sum(costs)/len(costs)
        mina, mintlo, minthi = a, thresholds[m], thresholds[n]


th_lo = mintlo
th_hi = minthi
print('threshold_lo =', th_lo)
print('threshold_hi =', th_hi)

threshold_lo = 7
threshold_hi = 9


### Problem 5 (1 pt)

Write code that prints out the average time (in seconds) that the volunteer `f1` spends actively brushing her teeth according to:
*   the ground truth
*   the `brush_indicator()` model with parameters you found in Problem 2
*   the `brush_indicator2()` model with parameters you found in Problem 4

In [6]:
## EDIT THE CODE IN THIS CELL

avg_time_f1_ground_truth = 0
avg_time_f1_brush_indicator = 0
avg_time_f1_brush_indicator2 = 0
freq = 32
files = [f for f in filenames if f[-6:] == "f1.txt"]
gtfiles = [g for g in sorted(os.listdir(ground_truth_path)) if g[-6:] == "f1.txt"]
for gt in gtfiles:
  avg_time_f1_ground_truth += sum(np.genfromtxt(ground_truth_path+gt).astype(float)) / freq
avg_time_f1_ground_truth /= len(gtfiles)
for f in files:
  avg_time_f1_brush_indicator += sum(brush_indicator(f, mina, mint)) / freq
avg_time_f1_brush_indicator /= len(files)
for f in files:
  avg_time_f1_brush_indicator2 += sum(brush_indicator2(f, mina, mintlo, minthi)) / freq
avg_time_f1_brush_indicator2 /= len(files)

print('Average brushing time for f1 (ground truth) =', 
      avg_time_f1_ground_truth, 'seconds')
print('Average brushing time for f1 (brush_indicator) =', 
      avg_time_f1_brush_indicator, 'seconds')
print('Average brushing time for f1 (brush_indicator2) =', 
      avg_time_f1_brush_indicator2, 'seconds')

Average brushing time for f1 (ground truth) = 44.465625 seconds
Average brushing time for f1 (brush_indicator) = 44.271875 seconds
Average brushing time for f1 (brush_indicator2) = 44.54375 seconds
